Unofficial Notes:
- The geometric mean is rarely significantly different from the arithmetic mean (>90% on average)
- The vast majority of SCV values are below 0.1

scv:
- count  1.118043e+08
- mean   1.085391e-01
- std    2.260635e-01
- min    3.887581e-05
- 25%    2.936568e-02
- 50%    5.386058e-02
- 75%    1.048740e-01
- max    5.767072e+01

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import sklearn

In [8]:
import re
import os
import math

In [9]:
starting_year = 2016
ending_year = 2017#2020
starting_quarter = 1
ending_quarter = 3

In [10]:
directory = "/home/matts/phd/data/uber/"

In [11]:
file_formats = {
    "weekday": directory + "hourly_agg/london-lsoa-{}-{}-OnlyWeekdays-HourlyAggregate.csv",
    "all":     directory + "hourly_agg/london-lsoa-{}-{}-All-HourlyAggregate.csv"
}

In [12]:
def get_aggregate_df(period):
    full_df = pd.DataFrame()
    
    def append_to_df(yr, qtr, full_df):
        qtr_df = pd.read_csv(file_formats[period].format(yr, qtr))
        qtr_df["Year"] = yr
        qtr_df["Quarter"] = qtr
        full_df = pd.concat([full_df, qtr_df])
        return full_df
        
    for yr in range(starting_year+1, ending_year):
        for qtr in range(1, 5):
            full_df = append_to_df(yr, qtr, full_df)
    for qtr in range(starting_quarter,5):
        full_df = append_to_df(starting_year, qtr, full_df)
    for qtr in range(1, ending_quarter+1):
        full_df = append_to_df(ending_year, qtr, full_df)
    
    return full_df

In [13]:
agg_df = get_aggregate_df("weekday")

In [15]:
agg_df

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Year,Quarter
0,1,2,8,1229.54,394.37,1173.73,1.35,2016,1
1,1,4,6,869.39,234.42,839.09,1.30,2016,1
2,2,1,18,1277.50,594.75,1159.71,1.58,2016,1
3,2,3,16,484.39,281.77,428.79,1.61,2016,1
4,2,4,5,129.09,143.18,98.77,1.89,2016,1
...,...,...,...,...,...,...,...,...,...
6813540,982,977,8,1013.78,306.55,980.64,1.28,2017,3
6813541,982,978,7,833.98,214.98,813.72,1.23,2017,3
6813542,982,979,6,817.54,218.34,796.08,1.24,2017,3
6813543,982,980,14,144.08,116.90,111.59,2.18,2017,3


In [18]:
def fit_log_normal(mean, std):
    exp_sig2_m1 = (std**2)/(math.e**(2*math.e**mean))
    sig2 = (math.log(exp_sig2_m1+1))
    mu = math.log((math.e**mean)+(sig2/2))
    return [mu, sig2**(1/2)]
def log_normal_gmean(params):
    return math.exp(params[0])

In [20]:
agg_df["ghat"] = agg_df.apply(lambda x: log_normal_gmean(fit_log_normal(x["mean_travel_time"], x["standard_deviation_travel_time"])),axis=1)

/tmp/ipykernel_30353/1027189982.py:2: RuntimeWarning: overflow encountered in double_scalars
  exp_sig2_m1 = (std**2)/(math.e**(2*math.e**mean))
/tmp/ipykernel_30353/1027189982.py:4: RuntimeWarning: overflow encountered in double_scalars
  mu = math.log((math.e**mean)+(sig2/2))


In [22]:
ggh = agg_df[["geometric_mean_travel_time","ghat"]]

In [24]:
agg_df.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Year,Quarter,ghat
0,1,2,8,1229.54,394.37,1173.73,1.35,2016,1,inf
1,1,4,6,869.39,234.42,839.09,1.30,2016,1,inf
2,2,1,18,1277.50,594.75,1159.71,1.58,2016,1,inf
3,2,3,16,484.39,281.77,428.79,1.61,2016,1,2.332943e+210
4,2,4,5,129.09,143.18,98.77,1.89,2016,1,1.156311e+56
